# Router

A HTTP Request router for coil

In [1]:
@require "." @route Request Response;

## API

### `@route(block::Expr)`

Creates a new root `Router`, puts it in focus, and evaluates the expression. Inside the expression you should put some `@route(::Expr, ::String)` expressions. These will define routes on the `Router`. This form returns a function which dispatches HTTP requests to the routes defined on it.

### `@route(handler::Expr, path::String)`

Defines a new route on the in focus `Router`. The `handler` expression will be transformed into a generic method, enabling it to take advantage of Julia's dispatch mechanism to select the type of HTTP request you want the handler to respond to. Also any parameters defined in the path will be coerced to the types declared in the handler.

In [2]:
route = @route begin
  @route("user/:(\\d+)") do r::Request{:GET}, id::Int
    Response(200, "getting user #$id")
  end

  @route("user/:(\\d+)") do r::Request{:PUT}, id::Int
    Response(200, "putting user #$id")
  end
end

(anonymous function)

The returned function then just takes a `Request` object and dispatches to the correct handler based on the Uri path contained within the `Request`. The handler will then presumably return a HTTP `Response` its not enforced

In [3]:
route(Request(IOBuffer("GET /user/1")))

Response(200,Dict("Content-Length"=>"15"),"getting user #1")

In [4]:
route(Request(IOBuffer("PUT /user/1")))

Response(200,Dict("Content-Length"=>"15"),"putting user #1")

### Error cases

Invalid HTTP method results in a `405`

In [5]:
route(Request(IOBuffer("DELETE /user/1")))

Response(405,Dict("Allow"=>"GET, PUT"),nothing)

Unless it was an unhandled OPTIONS request. In which case it will be a `204`

In [6]:
route(Request(IOBuffer("OPTIONS /user/1")))

Response(204,Dict("Allow"=>"GET, PUT"),nothing)

An incorrect path will result in a `404` response and it will include a somewhat helpful message in the body

In [7]:
route(Request(IOBuffer("GET /user")))

Response(405,Dict("Allow"=>"GET, PUT"),nothing)

In [8]:
route(Request(IOBuffer("GET /users")))

Response(404,Dict("Content-Length"=>"12"),"invalid path")